In [1]:
import os
import pandas as pd
from rich.console import Console
from rich.logging import RichHandler
from rich import pretty
import logging
import warnings
import subprocess
import hvac
from cryptography.fernet import Fernet
import base64

DATAFILE="/usr/src/working/data/Questions.xlsx"
ENCRYPTEDFILE="/usr/src/working/data/Questions_encrypted.xlsx"
KEYFILE="/usr/src/working/data/Questions.key"
WORKINGFILE="/usr/src/working/data/Questions_working.xlsx"

MODELFILE="/usr/src/working/models/modelfile"
MODELCONTEXT="/usr/src/working/trained"

# Configure logging
logging.disable(logging.WARNING) # disable warnings in the logger
warnings.filterwarnings("ignore") # disable warnings in warnings

logging.basicConfig(level=logging.INFO, format="%(message)s",handlers=[RichHandler()])
console = Console()

Load and decrypt the datafile

Encrypt and push the trainied model

In [ ]:
client = hvac.Client(
    url=os.environ["VAULT_ADDR"]
)

client.auth.userpass.login(
    username="datascience",
    password="password",
)

subprocess.run([
    "kapsule", "build",
    "--encryption-vault-addr", os.environ["VAULT_ADDR"],
    "--encryption-vault-auth-token", client.token,
    "--encryption-vault-key", "llm",
    "--encryption-vault-path", "transit",
    "--file", MODELFILE,
    "--tag", "docker.io/nicholasjackson/hug:enc",
    "--username", os.environ["DOCKER_USERNAME"],
    "--password", os.environ["DOCKER_PASSWORD"],
    MODELCONTEXT
])

In [ ]:
OLLAMA="/usr/src/working/ollama"

client.auth.userpass.login(
    username="runtime",
    password="password",
)

subprocess.run([
    "kapsule", "pull",
    "--encryption-vault-addr", os.environ["VAULT_ADDR"],
    "--encryption-vault-auth-token", client.token,
    "--encryption-vault-key", "llm",
    "--encryption-vault-path", "transit",
    "--format", "ollama",
    "--output", OLLAMA,
    "--username", os.environ["DOCKER_USERNAME"],
    "--password", os.environ["DOCKER_PASSWORD"],
    "docker.io/nicholasjackson/hug:enc"
])